### Dữ liệu được download tại: https://github.com/duyvuleo/VNTC
### Mô hình pre-trained được download tại: https://github.com/Kyubyong/wordvectors

In [1]:
import os
from gensim.models import KeyedVectors 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join(dir_path, "data/vi.vec")
print(word2vec_model_path)

w2v = KeyedVectors.load_word2vec_format(word2vec_model_path)
vocab = w2v.wv.vocab
wv = w2v.wv

/media/npthanh/New Volume/CTU/Spring-2020/NLP/Code/Natual-Language-Processing/Data/vi.vec


/home/npthanh/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [2]:
def word_vector(word):
    return wv[word]

In [3]:
topic = ['giao_thông', 'bóng_đá', 'tài_chính', 'thị_trường', 'sức_khoẻ', 'thế_giới', 'thể_thao', 'ẩm_thực']

In [4]:
list_word = ['đường_bộ', 'du_lịch', 'ông', 'đau', 'tai_nạn']

In [5]:
import numpy as np
def get_best_topic(list_word, list_topic):
    best_score = 0
    
    for topic in list_topic:
        topic_score = 0
        for word in list_word:
            score = wv.similarity(word, topic)
            topic_score += score
        
        if topic_score > best_score:
            best_topic = topic
            best_score = topic_score
    
    return best_topic

In [6]:
get_best_topic(list_word, topic)

/home/npthanh/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'giao_thông'

##  Test CNN

In [7]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from tensorflow.python.keras.models import Sequential, load_model

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *

Using TensorFlow backend.


In [8]:
from pyvi import ViTokenizer, ViPosTagger
from tqdm import tqdm
import numpy as np
import gensim
import numpy as np

# Tiền sử lý dữ liệu

In [9]:
def preprocessing_doc(doc):
    lines = gensim.utils.simple_preprocess(doc)
    lines = ' '.join(lines)
    lines = ViTokenizer.tokenize(lines)

    return lines

In [10]:
import pickle

X_data = pickle.load(open('data/X_data.pkl', 'rb'))
y_data = pickle.load(open('data/y_data.pkl', 'rb'))

X_test = pickle.load(open('data/X_test.pkl', 'rb'))
y_test = pickle.load(open('data/y_test.pkl', 'rb'))

In [11]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)

#### Transform by SVD to decrease number of dimensions

#### Word Level

In [14]:
from sklearn.decomposition import TruncatedSVD

In [15]:
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
             random_state=42, tol=0.0)

In [16]:
X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

In [24]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

In [33]:
number_to_class = encoder.classes_

In [34]:
number_to_class

array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

### Huấn luyện mô hình

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))

In [21]:
from keras.layers import *

In [22]:
def create_cnn_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu', return_sequences=True))(layer)    
    layer = Convolution1D(100, 3, activation="relu")(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.summary()
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [25]:
classifier = create_cnn_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=20)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300)               0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 10, 30)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 256)           122112    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 8, 100)            76900     
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               410112    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               2626

In [44]:
test_doc = '''
Cơ quan Cảnh sát điều tra (Bộ Công an) ngày 18/3 ra kết luận điều tra, đề nghị truy tố 12 bị can trong vụ án xảy ra tại Ngân hàng TMCP Đầu tư và Phát triển Việt Nam (BIDV) và một số công ty.

Kết luận điều tra xác định, ông Hà giữ chức Chủ tịch HĐQT BIDV giai đoạn 2008-2016, đại diện 40% vốn Nhà nước tại ngân hàng này. Khi đương nhiệm, ông Hà đã vi phạm quy chế làm việc của BIDV, có biểu hiện áp đặt, thiếu dân chủ trong chỉ đạo. Ông lấy danh nghĩa BIDV trực tiếp thực hiện các hoạt động xúc tiến đầu tư tại tỉnh Hà Tĩnh cho công ty sân sau trái quy định.

Ông Hà chỉ đạo và phê duyệt cấp tín dụng với các điều kiện ưu đãi trong khi hai công ty sân sau là Công ty Cổ phần chăn nuôi Bình Hà và Tập đoàn An Phú không đủ năng lực tài chính về vốn tự có và năng lực thực hiện dự án chăn nuôi bò tại Hà Tĩnh. Lãnh đạo hai công ty sau khi được phê duyệt cho vay và nhận tiền giải ngân đã chiếm đoạt để sử dụng cá nhân hoặc sử dụng vốn vay không đúng mục đích. Dự án liên tục thua lỗ phải dừng hoạt động, gây thiệt hại hơn 1.500 tỷ đồng cho BIDV.

Hành vi của ông Hà phạm vào tội Vi phạm quy định về hoạt động ngân hàng, hoạt động khác liên quan đến hoạt động ngân hàng, theo điều 206 Bộ luật Hình sự. Tuy nhiên, tháng 7/2019, ông Hà tử vong về bệnh lý khi đang tạm giam nên Cơ quan cảnh sát điều tra (Bộ Công an) đã đình chỉ điều tra bị can.

Các bị can Trần Lục Lang và Đoàn Ánh Sáng (đều là cựu phó tổng giám đốc BIDV), Kiều Đình Hòa (cựu giám đốc BIDV chi nhánh Hà Tĩnh), Lê Thị Vân Anh (cựu trưởng phòng khách hàng doanh nghiệp BIDV chi nhánh Hà Tĩnh) đã đồng phạm với ông Trần Bắc Hà trong việc thẩm định, cấp tín dụng cho Công ty Bình Hà gây thiệt hại cho BIDV hơn 683 tỷ đồng.
'''

In [45]:
test_doc = preprocessing_doc(test_doc)

In [46]:
test_doc_tfidf = tfidf_vect.transform([test_doc])
print(np.shape(test_doc_tfidf))
test_doc_svd = svd.transform(test_doc_tfidf)

(1, 30000)


In [47]:
probabilities = classifier.predict(test_doc_svd)

In [48]:
index = np.argsort(probabilities[0,:])

In [49]:
print("Most likely class:", number_to_class[index[9]], "-- Probability:", probabilities[0,index[9]])
print("Second most likely class:", number_to_class[index[8]], "-- Probability:", probabilities[0,index[8]])
print("Third most likely class:", number_to_class[index[7]], "-- Probability:", probabilities[0,index[7]])
print("Fourth most likely class:", number_to_class[index[6]], "-- Probability:", probabilities[0,index[6]])
print("Fifth most likely class:", number_to_class[index[5]], "-- Probability:", probabilities[0,index[5]])

Most likely class: Phap luat -- Probability: 0.9946531
Second most likely class: Chinh tri Xa hoi -- Probability: 0.0046064435
Third most likely class: Kinh doanh -- Probability: 0.00055128935
Fourth most likely class: Doi song -- Probability: 0.00012419866
Fifth most likely class: The thao -- Probability: 4.5744415e-05


In [37]:
classifier.save('CNN_Text_Classifier.h5') # Save model for using later